# MMOB Project

# import

in this cell, relevant libraries are imported, as well as the datafile imported from our github and a dict that will store all results during the project 

In [229]:
import pandas as pd
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, Variable
from biogeme import models
from biogeme import results as res
from biogeme.expressions import DefineVariable, log
data_file ='https://raw.githubusercontent.com/DunodMax/MMOB/main/Dataset.txt'
LPMC = pd.read_csv(data_file, sep='\t')
LPMC
database = db.Database('LPMC', LPMC)
all_results = {}
from collections import namedtuple

# Model 0

all useful columns for this model are used as variables

In [176]:
travel_mode=Variable('travel_mode')
dur_pt_access=Variable('dur_pt_access')
dur_pt_rail=Variable('dur_pt_rail')
dur_pt_bus=Variable('dur_pt_bus')
dur_pt_int=Variable('dur_pt_int')
pt_interchanges=Variable('pt_interchanges')
dur_driving=Variable('dur_driving')
cost_transit=Variable('cost_transit')
cost_driving_fuel=Variable('cost_driving_fuel')
cost_driving_ccharge=Variable('cost_driving_ccharge')
dur_walking=Variable('dur_walking')
dur_cycling=Variable('dur_cycling')
female=Variable('female')
age=Variable('age')

Trip duration for each mode are computed ( see report for details about equations)
Cost for each mode with a cost are computed ( cycling and walking is considered free)

In [69]:
time_walk=dur_walking
time_cycle=dur_cycling
time_pt=dur_pt_access+dur_pt_rail+dur_pt_bus+dur_pt_int
time_drive=dur_driving

cost_drive=cost_driving_ccharge+cost_driving_fuel
cost_pt=cost_transit

There is 4 different travel mode, we will thus build a model with 4 utility functions, we create 3 alternative specific constant, a generic parameter for travel time and a generic parameter for cost

In [70]:
asc_cycle = Beta('asc_cycle', 0, None, None, 0)
asc_pt = Beta('asc_pt', 0, None, None, 0)
asc_drive = Beta('asc_drive', 0, None, None, 0)
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time = Beta('beta_time', 0, None, None, 0)

utility functions

In [71]:
v_walk_model0= beta_time * time_walk  
v_cycle_model0= asc_cycle + beta_time * time_cycle 
v_pt_model0= asc_pt + beta_time * time_pt + beta_cost * cost_pt
v_drive_model0= asc_drive + beta_time * time_drive + beta_cost * cost_drive

In [72]:
av = {1: 1, 2: 1, 3: 1, 4:1}

estimation of the parameters for Model 0, all relevant parameters are printed

In [73]:
V_model0 = {1: v_walk_model0 , 2: v_cycle_model0, 3: v_pt_model0, 4: v_drive_model0}
logprob_model0 = models.loglogit(V_model0, av, travel_mode)
biogeme_model0 = bio.BIOGEME(database, logprob_model0)
biogeme_model0.modelName = 'Model_0'
all_results['Model0'] = biogeme_model0.estimate()
results_generic = biogeme_model0.estimate()
results_generic.getEstimatedParameters()

[20:55:18] < Warning >   Cannot read file __Model_0.iter. Statement is ignored.


,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-3.832163,0.107574,-35.623351,0.0
asc_drive,-1.235707,0.080400,-15.369411,0.0
asc_pt,-0.528767,0.054766,-9.655006,0.0
beta_cost,-0.169780,0.013108,-12.952753,0.0
beta_time,-5.450540,0.202684,-26.891767,0.0


In [74]:
res.compileEstimationResults(all_results)

,Model0
Number of estimated parameters,5.000000
Sample size,5000.000000
Final log likelihood,-4566.446492
Akaike Information Criterion,9142.892983
Bayesian Information Criterion,9175.478949
asc_cycle,-3.832163
asc_drive,-1.235707
asc_pt,-0.528767
beta_cost,-0.169780
beta_time,-5.450540


In [75]:
print("Null Loglikelihood : ")
biogeme_model0.calculateNullLoglikelihood(av)

Null Loglikelihood : 


-6931.471805599917

# Model 1 

### Time specification

We will try a specification where the the cost coeffecient is Generic and the time coefficient is alternative specific

Again 4 different travel mode, 4 utility functions and 3 alternative specific constant.
This time we have the generic parameter for cost and a 4 new specific parameter for travel time

In [76]:
beta_time_drive = Beta('beta_time_drive', 0, None, None, 0)
beta_time_pt = Beta('beta_time_pt', 0, None, None, 0)
beta_time_walk = Beta('beta_time_walk', 0, None, None, 0)
beta_time_cycle = Beta('beta_time_cycle', 0, None, None, 0)

New utility functions :

In [77]:
v_walk_model1 = beta_time_walk * time_walk  
v_cycle_model1 = asc_cycle + beta_time_cycle * time_cycle 
v_pt_model1 = asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model1 = asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive

Estimation of the parameters for Model 1, all relevant parameters are printed

In [78]:
V_model1 = {1: v_walk_model1 , 2: v_cycle_model1, 3: v_pt_model1, 4: v_drive_model1}
logprob_model1 = models.loglogit(V_model1, av, travel_mode)
biogeme_model1 = bio.BIOGEME(database, logprob_model1)
biogeme_model1.modelName = 'Model1_time_specification'
all_results['Model1_time_specification'] = biogeme_model1.estimate()
results_alt_spec_time = biogeme_model1.estimate()
results_alt_spec_time.getEstimatedParameters()

[20:55:19] < Warning >   Cannot read file __Model1_time_specification.iter. Statement is ignored.


,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.509593,0.191028,-23.606921,0.0
asc_drive,-1.885701,0.122740,-15.363361,0.0
asc_pt,-2.339709,0.125798,-18.598939,0.0
beta_cost,-0.146061,0.014850,-9.835754,0.0
beta_time_cycle,-6.121448,0.542680,-11.280025,0.0
beta_time_drive,-6.445066,0.391345,-16.469010,0.0
beta_time_pt,-3.514554,0.258207,-13.611367,0.0
beta_time_walk,-8.204816,0.364365,-22.518121,0.0


In [79]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification
Number of estimated parameters,5,8
Sample size,5000,5000
Final log likelihood,-4566.446492,-4264.834249
Akaike Information Criterion,9142.892983,8545.668498
Bayesian Information Criterion,9175.478949,8597.806043
asc_cycle,-3.832163,-4.509598
asc_drive,-1.235707,-1.885704
asc_pt,-0.528767,-2.339712
beta_cost,-0.16978,-0.146061
beta_time,-5.45054,


### Cost specification

We compare with the model where we specify the price, to be sure to have the most interesting model for the future models. (walk and cycling don't have cost, so only 2 specific parameter for cost)

In [80]:
beta_cost_drive = Beta('beta_cost_drive', 0, None, None, 0)
beta_cost_pt = Beta('beta_cost_pt', 0, None, None, 0)

New utility function

In [81]:
v_walk_model1_spec_cost = beta_time * time_walk  
v_cycle_model1_spec_cost = asc_cycle + beta_time * time_cycle 
v_pt_model1_spec_cost = asc_pt + beta_time * time_pt + beta_cost_pt * cost_pt
v_drive_model1_spec_cost = asc_drive + beta_time * time_drive + beta_cost_drive * cost_drive

In [82]:
V_model1_spec_cost = {1: v_walk_model1_spec_cost , 2: v_cycle_model1_spec_cost, 3: v_pt_model1_spec_cost, 4: v_drive_model1_spec_cost}
logprob_model1_spec_cost = models.loglogit(V_model1_spec_cost, av, travel_mode)
biogeme_model1_spec_cost = bio.BIOGEME(database, logprob_model1_spec_cost)
biogeme_model1_spec_cost.modelName = 'Model_1_cost_specifiaction'
all_results['Model1_cost_specification'] = biogeme_model1_spec_cost.estimate()
results_alt_spec_cost = biogeme_model1_spec_cost.estimate()
results_alt_spec_cost.getEstimatedParameters()

[20:55:20] < Warning >   Cannot read file __Model_1_cost_specifiaction.iter. Statement is ignored.


,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-3.766977,0.107734,-34.965548,0.000000
asc_drive,-1.137426,0.080194,-14.183498,0.000000
asc_pt,-0.831978,0.064856,-12.828129,0.000000
beta_cost_drive,-0.214644,0.021272,-10.090610,0.000000
beta_cost_pt,0.061554,0.028466,2.162411,0.030587
beta_time,-5.411681,0.202913,-26.670010,0.000000


In [83]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification,Model1_cost_specification
Number of estimated parameters,5,8,6
Sample size,5000,5000,5000
Final log likelihood,-4566.446492,-4264.834249,-4509.365231
Akaike Information Criterion,9142.892983,8545.668498,9030.730462
Bayesian Information Criterion,9175.478949,8597.806043,9069.833621
asc_cycle,-3.832163,-4.509598,-3.766977
asc_drive,-1.235707,-1.885704,-1.137426
asc_pt,-0.528767,-2.339712,-0.831978
beta_cost,-0.16978,-0.146061,
beta_time,-5.45054,,-5.411681


### Comparaison

we use log-likehood ratio as the model 0 is a reduction of the model 1

In [84]:
old  = results_generic
new  = results_alt_spec_time
new.likelihood_ratio_test(old, 0.05) #level of the statistics for a level of significance of 5%?

LRTuple(message='H0 can be rejected at level 5.0%', statistic=603.2244866325364, threshold=7.814727903251179)

# Model 2

In [85]:
LPMC.columns

Index(['trip_id', 'household_id', 'person_n', 'trip_n', 'travel_mode',
       'purpose', 'fueltype', 'faretype', 'bus_scale', 'survey_year',
       'travel_year', 'travel_month', 'travel_date', 'day_of_week',
       'start_time', 'age', 'female', 'driving_license', 'car_ownership',
       'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access',
       'dur_pt_rail', 'dur_pt_bus', 'dur_pt_int', 'pt_interchanges',
       'dur_driving', 'cost_transit', 'cost_driving_fuel',
       'cost_driving_ccharge', 'driving_traffic_percent'],
      dtype='object')

In [86]:
model_base=biogeme_model1

In [87]:
#we're adding the attribute distance to our model 
#we're adding the interaction between age and distance
pt_interchanges=Variable('pt_interchanges')
driving_license=Variable('driving_license')

In [88]:
#defining associated coefficients
beta_pt_inter= Beta('beta_pt_inter', 0, None, None, 0)
beta_license= Beta('beta_license', 0, None, None, 0)

In [89]:
#the specification, we only consider the alternative distance for walking
#the interaction distance age is considered just for public trasnport
v_walk_model2 = v_walk_model1
v_cycle_model2 = v_cycle_model1
v_pt_model2 = v_pt_model1 + beta_pt_inter*pt_interchanges
v_drive_model2 = v_drive_model1 + asc_drive*beta_license*driving_license

In [90]:
#the estimation results
V_model2 = {1: v_walk_model2 , 2: v_cycle_model2, 3: v_pt_model2, 4: v_drive_model2}
logprob_model2 = models.loglogit(V_model2, av, travel_mode)
biogeme_model2 = bio.BIOGEME(database, logprob_model2)
biogeme_model2.modelName = 'Model_2'
all_results['Model_2'] = biogeme_model2.estimate()
results_model2 = biogeme_model2.estimate()
results_model2.getEstimatedParameters()

[20:55:21] < Warning >   Cannot read file __Model_2.iter. Statement is ignored.


,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.566489,0.196947,-23.186435,0.000000
asc_drive,-2.782432,0.138492,-20.090922,0.000000
asc_pt,-2.421769,0.131403,-18.430079,0.000000
beta_cost,-0.140456,0.013889,-10.112732,0.000000
beta_license,-0.517648,0.027952,-18.519340,0.000000
beta_pt_inter,-0.107552,0.089736,-1.198541,0.230707
beta_time_cycle,-6.305828,0.559567,-11.269120,0.000000
beta_time_drive,-7.003842,0.411151,-17.034715,0.000000
beta_time_pt,-3.541166,0.282515,-12.534425,0.000000
beta_time_walk,-8.376052,0.377032,-22.215752,0.000000


In [91]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification,Model1_cost_specification,Model_2
Number of estimated parameters,5,8,6,10
Sample size,5000,5000,5000,5000
Final log likelihood,-4566.446492,-4264.834249,-4509.365231,-4039.286551
Akaike Information Criterion,9142.892983,8545.668498,9030.730462,8098.573101
Bayesian Information Criterion,9175.478949,8597.806043,9069.833621,8163.745033
asc_cycle,-3.832163,-4.509598,-3.766977,-4.566495
asc_drive,-1.235707,-1.885704,-1.137426,-2.782437
asc_pt,-0.528767,-2.339712,-0.831978,-2.421776
beta_cost,-0.16978,-0.146061,,-0.140457
beta_time,-5.45054,,-5.411681,


In [92]:
#we test base_model against model_2 where base_model is a restricted version of model_2 suing the looglikehood
#ratio test

In [93]:
#we test them using the null hypothesis that our base model is the true model

In [94]:
results_base=model_base.estimate()

In [95]:
results_model2.likelihood_ratio_test(results_base, 0.05) #level of the statistics for a level of significance of 5%?

LRTuple(message='H0 can be rejected at level 5.0%', statistic=451.09539568599666, threshold=5.991464547107979)

In [96]:
#then depending on if the value I obtained was higher or lower than a certain number, we reject or accept the null hypothesis

In [97]:
from scipy.stats import chi2
threshold = chi2.ppf(.95, 2)
threshold #just in case we might want to compare to this threshold

5.991464547107979

# Model 3

We will conduct a non-linear transformation of time

In [98]:
model_base=biogeme_model2
log_time_walk = database.DefineVariable('log_time_walk', log(time_walk))
log_time_cycle = database.DefineVariable('log_time_cycle', log(time_cycle))
log_time_pt = database.DefineVariable('log_time_pt', log(time_pt))
log_time_drive = database.DefineVariable('log_time_drive', log(time_drive))
log_beta_time_drive = Beta('log_beta_time_drive', 0, None, None, 0)
log_beta_time_pt = Beta('log_beta_time_pt', 0, None, None, 0)
log_beta_time_walk = Beta('log_beta_time_walk', 0, None, None, 0)
log_beta_time_cycle = Beta('log_beta_time_cycle', 0, None, None, 0)

In [99]:
v_walk_model3 = log_beta_time_walk * log_time_walk
v_cycle_model3 = asc_cycle + log_beta_time_cycle * log_time_cycle
v_pt_model3 = asc_pt + log_beta_time_pt * log_time_pt + beta_cost * cost_pt + beta_pt_inter*pt_interchanges
v_drive_model3 = asc_drive + log_beta_time_drive * log_time_drive + beta_cost * cost_drive + asc_drive*beta_license*driving_license

In [100]:
V_model3 = {1: v_walk_model3 , 2: v_cycle_model3, 3: v_pt_model3, 4: v_drive_model3}
logprob_model3 = models.loglogit(V_model3, av, travel_mode)
biogeme_model3 = bio.BIOGEME(database, logprob_model3)
biogeme_model3.modelName = 'Model_3'
all_results['Model_3'] = biogeme_model3.estimate()
results_model3 = biogeme_model3.estimate()
results_model3.getEstimatedParameters()

[20:55:23] < Warning >   Cannot read file __Model_3.iter. Statement is ignored.


,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-2.440357,0.227036,-10.748772,0.000000
asc_drive,-0.680104,0.180008,-3.778185,0.000158
asc_pt,1.677853,0.147456,11.378653,0.000000
beta_cost,-0.137105,0.013208,-10.380838,0.000000
beta_license,-2.101083,0.537274,-3.910636,0.000092
beta_pt_inter,-0.093153,0.073977,-1.259214,0.207953
log_beta_time_cycle,-2.464649,0.131859,-18.691558,0.000000
log_beta_time_drive,-2.344192,0.101472,-23.101947,0.000000
log_beta_time_pt,-2.025236,0.120497,-16.807409,0.000000
log_beta_time_walk,-4.598998,0.133120,-34.547665,0.000000


In [101]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification,Model1_cost_specification,Model_2,Model_3
Number of estimated parameters,5,8,6,10,10
Sample size,5000,5000,5000,5000,5000
Final log likelihood,-4566.446492,-4264.834249,-4509.365231,-4039.286551,-3975.362089
Akaike Information Criterion,9142.892983,8545.668498,9030.730462,8098.573101,7970.724177
Bayesian Information Criterion,9175.478949,8597.806043,9069.833621,8163.745033,8035.896109
asc_cycle,-3.832163,-4.509598,-3.766977,-4.566495,-2.440357
asc_drive,-1.235707,-1.885704,-1.137426,-2.782437,-0.680104
asc_pt,-0.528767,-2.339712,-0.831978,-2.421776,1.677853
beta_cost,-0.16978,-0.146061,,-0.140457,-0.137105
beta_time,-5.45054,,-5.411681,,


We will compare Model_2 and Model_3 with a cox test, thus we will create a generic model

In [102]:
v_walk_model_generic = log_beta_time_walk * log_time_walk + beta_time_walk * time_walk
v_cycle_model_generic = asc_cycle + log_beta_time_cycle * log_time_cycle + beta_time_cycle * time_cycle
v_pt_model_generic = asc_pt + log_beta_time_pt * log_time_pt + beta_cost * cost_pt + beta_pt_inter*pt_interchanges + beta_time_pt * time_pt
v_drive_model_generic = asc_drive + log_beta_time_drive * log_time_drive + beta_cost * cost_drive + asc_drive*beta_license*driving_license + beta_time_drive * time_drive

In [103]:
V_model_generic = {1: v_walk_model_generic , 2: v_cycle_model_generic, 3: v_pt_model_generic, 4: v_drive_model_generic}
logprob_model_generic = models.loglogit(V_model_generic, av, travel_mode)
biogeme_model_generic = bio.BIOGEME(database, logprob_model_generic)
biogeme_model_generic.modelName = 'Model_generic'
all_results['Model_generic'] = biogeme_model_generic.estimate()
results_model_generic = biogeme_model_generic.estimate()
results_model_generic.getEstimatedParameters()

[20:55:24] < Warning >   Cannot read file __Model_generic.iter. Statement is ignored.


,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-2.092631,0.676585,-3.092932,1.981897e-03
asc_drive,-0.311548,0.360765,-0.863575,3.878215e-01
asc_pt,2.500510,0.466756,5.357207,8.451816e-08
beta_cost,-0.129692,0.013880,-9.343881,0.000000e+00
beta_license,-4.630841,5.359639,-0.864021,3.875764e-01
beta_pt_inter,-0.035905,0.085233,-0.421256,6.735685e-01
beta_time_cycle,-2.579461,0.793375,-3.251252,1.148979e-03
beta_time_drive,-3.075970,0.576765,-5.333139,9.652933e-08
beta_time_pt,-2.944321,0.534183,-5.511823,3.551369e-08
beta_time_walk,-1.610458,0.464996,-3.463383,5.334294e-04


In [104]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification,Model1_cost_specification,Model_2,Model_3,Model_generic
Number of estimated parameters,5,8,6,10,10,14
Sample size,5000,5000,5000,5000,5000,5000
Final log likelihood,-4566.446492,-4264.834249,-4509.365231,-4039.286551,-3975.362089,-3948.931227
Akaike Information Criterion,9142.892983,8545.668498,9030.730462,8098.573101,7970.724177,7925.862453
Bayesian Information Criterion,9175.478949,8597.806043,9069.833621,8163.745033,8035.896109,8017.103158
asc_cycle,-3.832163,-4.509598,-3.766977,-4.566495,-2.440357,-2.092628
asc_drive,-1.235707,-1.885704,-1.137426,-2.782437,-0.680104,-0.311544
asc_pt,-0.528767,-2.339712,-0.831978,-2.421776,1.677853,2.500508
beta_cost,-0.16978,-0.146061,,-0.140457,-0.137105,-0.129692
beta_time,-5.45054,,-5.411681,,,


In [105]:
model_base=biogeme_model2
results_base=model_base.estimate()
results_model_generic.likelihood_ratio_test(results_base, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=180.71064703482534, threshold=9.487729036781154)

In [106]:
model_base=biogeme_model3
results_base=model_base.estimate()
results_model_generic.likelihood_ratio_test(results_base, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=52.86172453334257, threshold=9.487729036781154)

In this case the cox-test tells us to find better models, we will thus use a box-cox transform

In [107]:
lambda_boxcox = Beta('lambda_boxcox', 1, None, None, 0)
boxcox_time_walk = models.boxcox(time_walk, lambda_boxcox)
boxcox_time_cycle = models.boxcox(time_cycle, lambda_boxcox)
boxcox_time_pt = models.boxcox(time_pt, lambda_boxcox)
boxcox_time_drive = models.boxcox(time_drive , lambda_boxcox)
beta_elapsed_time_walk = Beta('beta_elapsed_walk', 0, None, None, 0)
beta_elapsed_time_cycle = Beta('beta_elapsed_cycle', 0, None, None, 0)
beta_elapsed_time_pt = Beta('beta_elapsed_time_pt', 0, None, None, 0)
beta_elapsed_time_drive = Beta('beta_elapsed_time_drive', 0, None, None, 0)
v_walk_model_boxcox = beta_elapsed_time_walk * boxcox_time_walk
v_cycle_model_boxcox  = asc_cycle + beta_elapsed_time_cycle * boxcox_time_cycle 
v_pt_model_boxcox  = asc_pt + beta_elapsed_time_pt * boxcox_time_pt   + beta_cost * cost_pt + beta_pt_inter*pt_interchanges
v_drive_model_boxcox  = asc_drive + beta_elapsed_time_drive * boxcox_time_drive + beta_cost * cost_drive + asc_drive*beta_license*driving_license

In [108]:
V_model_boxcox = {1: v_walk_model_boxcox , 2: v_cycle_model_boxcox, 3: v_pt_model_boxcox, 4: v_drive_model_boxcox}
logprob_model_boxcox = models.loglogit(V_model_boxcox, av, travel_mode)
biogeme_model_boxcox = bio.BIOGEME(database, logprob_model_boxcox)
biogeme_model_boxcox.modelName = 'Model_boxcox'
all_results['Model_boxcox'] = biogeme_model_boxcox.estimate()
results_model_boxcox = biogeme_model_boxcox.estimate()
results_model_boxcox.getEstimatedParameters()

[20:55:30] < Warning >   Cannot read file __Model_boxcox.iter. Statement is ignored.


,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-2.794017,0.287787,-9.708621,0.000000e+00
asc_drive,-1.325820,0.237166,-5.590256,2.267351e-08
asc_pt,1.641993,0.177913,9.229193,0.000000e+00
beta_cost,-0.129339,0.013804,-9.370027,0.000000e+00
beta_elapsed_cycle,-3.661618,0.287125,-12.752709,0.000000e+00
beta_elapsed_time_drive,-3.701045,0.275373,-13.440099,0.000000e+00
beta_elapsed_time_pt,-2.718524,0.190871,-14.242719,0.000000e+00
beta_elapsed_walk,-5.667674,0.237039,-23.910320,0.000000e+00
beta_license,-1.091700,0.190782,-5.722244,1.051262e-08
beta_pt_inter,-0.147415,0.080668,-1.827429,6.763534e-02


In [109]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification,Model1_cost_specification,Model_2,Model_3,Model_generic,Model_boxcox
Number of estimated parameters,5,8,6,10,10,14,11
Sample size,5000,5000,5000,5000,5000,5000,5000
Final log likelihood,-4566.446492,-4264.834249,-4509.365231,-4039.286551,-3975.362089,-3948.931227,-3953.048852
Akaike Information Criterion,9142.892983,8545.668498,9030.730462,8098.573101,7970.724177,7925.862453,7928.097705
Bayesian Information Criterion,9175.478949,8597.806043,9069.833621,8163.745033,8035.896109,8017.103158,7999.78683
asc_cycle,-3.832163,-4.509598,-3.766977,-4.566495,-2.440357,-2.092628,-2.794017
asc_drive,-1.235707,-1.885704,-1.137426,-2.782437,-0.680104,-0.311544,-1.32582
asc_pt,-0.528767,-2.339712,-0.831978,-2.421776,1.677853,2.500508,1.641993
beta_cost,-0.16978,-0.146061,,-0.140457,-0.137105,-0.129692,-0.129339
beta_time,-5.45054,,-5.411681,,,,


In [110]:
model_base=biogeme_model2
results_base=model_base.estimate()
results_model_boxcox.likelihood_ratio_test(results_base, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=172.4753942531106, threshold=3.841458820694124)

# Model 4

In [111]:

model_base=biogeme_model_boxcox
V_nested=V_model_boxcox
#create the nests
Mu_motorized=Beta("Mu_motorized",1,0,None,0)
Mu_notmotorized=Beta("Mu_notmotorized",1,0,None,0)
motorized= Mu_motorized,[3,4]
notmotorized=Mu_notmotorized, [1,2]
nests_motor=motorized, notmotorized


In [112]:
logprob_model_nested = models.lognested(V_nested, av, nests_motor, travel_mode)
biogeme_model_nested = bio.BIOGEME(database, logprob_model_nested)
biogeme_model_nested.modelName = 'Model_nested_motor'
all_results['Model_nested_motor'] = biogeme_model_nested.estimate()
results_model_nested = biogeme_model_nested.estimate()
results_model_nested.getEstimatedParameters()

[20:55:41] < Warning >   Cannot read file __Model_nested_motor.iter. Statement is ignored.


,Value,Rob. Std err,Rob. t-test,Rob. p-value
Mu_motorized,1.574722,0.087201,18.058604,0.000000e+00
Mu_notmotorized,0.931015,0.118269,7.872037,3.552714e-15
asc_cycle,-2.116918,0.245672,-8.616852,0.000000e+00
asc_drive,-0.043311,0.006217,-6.966031,3.260059e-12
asc_pt,2.126026,0.114712,18.533543,0.000000e+00
beta_cost,-0.073239,0.011026,-6.642218,3.089973e-11
beta_elapsed_cycle,-2.598804,0.232892,-11.158851,0.000000e+00
beta_elapsed_time_drive,-2.576367,0.180690,-14.258493,0.000000e+00
beta_elapsed_time_pt,-1.938139,0.129231,-14.997482,0.000000e+00
beta_elapsed_walk,-4.784036,0.258992,-18.471760,0.000000e+00


This model is rejected as Mu_notmotorized is inferior to 1. We will try a crossnested model between motored and private.

In [113]:
#define the crossnests
Mu_private=Beta('Mu_private',3,1,None,0)
Mu_motorcross=Beta('Mu_Motorcross',3,1,None,0)
Alpha_Motor=Beta('Alpha_Motor',0.5,0,1,0)
Alpha_Private=1-Alpha_Motor

alpha_motor={1:0.0,
             2:0.0,
             3:Alpha_Motor,
             4:1.0 }
alpha_private={1:1.0,
               2:1.0,
               3:Alpha_Private,
               4:0.0}

nest_motor=Mu_motorcross, alpha_motor
nest_private=Mu_private, alpha_private
cross_nests=nest_private, nest_motor

In [114]:
logprob_model_crossnested = models.logcnl_avail(V_nested, av, cross_nests, travel_mode)
logprob_model_crossnested.changeInitValues(results_model_nested.getBetaValues())
biogeme_model_crossnested = bio.BIOGEME(database, logprob_model_crossnested)
biogeme_model_crossnested.modelName = 'Model_crossnested'
results_model_crossnested = biogeme_model_crossnested.estimate()
all_results['Model_crossnested'] = results_model_crossnested
results_model_crossnested.getEstimatedParameters()

[20:57:07] < Warning >   Cannot read file __Model_crossnested.iter. Statement is ignored.


,Value,Active bound,Rob. Std err,Rob. t-test,Rob. p-value
Alpha_Motor,1.000000,1.0,1.797693e+308,5.562685e-309,1.000000e+00
Mu_Motorcross,1.594238,0.0,8.956008e-02,1.780077e+01,0.000000e+00
Mu_private,1.000000,1.0,1.354415e-01,7.383260e+00,1.545430e-13
asc_cycle,-2.193791,0.0,2.541377e-01,-8.632292e+00,0.000000e+00
asc_drive,-0.042817,0.0,6.175418e-03,-6.933429e+00,4.107603e-12
asc_pt,2.094309,0.0,1.154743e-01,1.813658e+01,0.000000e+00
beta_cost,-0.072457,0.0,1.092374e-02,-6.632964e+00,3.290124e-11
beta_elapsed_cycle,-2.676142,0.0,2.340608e-01,-1.143354e+01,0.000000e+00
beta_elapsed_time_drive,-2.526094,0.0,1.814253e-01,-1.392360e+01,0.000000e+00
beta_elapsed_time_pt,-1.904853,0.0,1.296846e-01,-1.468835e+01,0.000000e+00


The model is also rejected as alpha is 1. Moreover we see that mu_private is 1. We try another simple nest with the use of a vehicule:

In [115]:
Mu_vehicle=Beta("Mu_vehicle",1,0,None,0)
vehicle= Mu_vehicle,[2,3,4]
not_vehicle=1,[1]
nests_vehicle=vehicle, not_vehicle


In [116]:
logprob_model_nestedvehicle = models.lognested(V_nested, av, nests_vehicle, travel_mode)
biogeme_model_nestedvehicle = bio.BIOGEME(database, logprob_model_nestedvehicle)
biogeme_model_nestedvehicle.modelName = 'Model_nested_vehicle'
results_model_nestedvehicle = biogeme_model_nestedvehicle.estimate()
all_results['Model_nested_vehicle'] = results_model_nestedvehicle
table_nestedvehicle=results_model_nestedvehicle.getEstimatedParameters()
table_nestedvehicle

[20:58:30] < Warning >   Cannot read file __Model_nested_vehicle.iter. Statement is ignored.


,Value,Rob. Std err,Rob. t-test,Rob. p-value
Mu_vehicle,1.469935,0.073312,20.050498,0.000000e+00
asc_cycle,-0.873915,0.189228,-4.618319,3.868613e-06
asc_drive,-0.041481,0.009402,-4.412101,1.023724e-05
asc_pt,2.127413,0.103051,20.644237,0.000000e+00
beta_cost,-0.082936,0.011377,-7.290078,3.097522e-13
beta_elapsed_cycle,-2.478241,0.181089,-13.685242,0.000000e+00
beta_elapsed_time_drive,-2.587127,0.155010,-16.690110,0.000000e+00
beta_elapsed_time_pt,-1.942677,0.113055,-17.183426,0.000000e+00
beta_elapsed_walk,-4.799870,0.216498,-22.170551,0.000000e+00
beta_license,-25.822391,5.762750,-4.480915,7.432379e-06


In [117]:
#t-test H0:true value is 1
mu = table_nestedvehicle.loc['Mu_vehicle', 'Value']
mu_stderr = table_nestedvehicle.loc['Mu_vehicle', 'Rob. Std err']
tested_value = 1
ttest = (mu - tested_value) / mu_stderr
ttest

6.410101266129394

In [118]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification,Model1_cost_specification,Model_2,Model_3,Model_generic,Model_boxcox,Model_nested_motor,Model_crossnested,Model_nested_vehicle
Number of estimated parameters,5,8,6,10,10,14,11,13,14,12
Sample size,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000
Final log likelihood,-4566.446492,-4264.834249,-4509.365231,-4039.286551,-3975.362089,-3948.931227,-3953.048852,-3927.355075,-3927.480361,-3939.746338
Akaike Information Criterion,9142.892983,8545.668498,9030.730462,8098.573101,7970.724177,7925.862453,7928.097705,7880.71015,7882.960722,7903.492676
Bayesian Information Criterion,9175.478949,8597.806043,9069.833621,8163.745033,8035.896109,8017.103158,7999.78683,7965.433661,7974.201427,7981.698995
asc_cycle,-3.832163,-4.509598,-3.766977,-4.566495,-2.440357,-2.092628,-2.794017,-2.116918,-2.193791,-0.873915
asc_drive,-1.235707,-1.885704,-1.137426,-2.782437,-0.680104,-0.311544,-1.32582,-0.043311,-0.042817,-0.041481
asc_pt,-0.528767,-2.339712,-0.831978,-2.421776,1.677853,2.500508,1.641993,2.126026,2.094309,2.127413
beta_cost,-0.16978,-0.146061,,-0.140457,-0.137105,-0.129692,-0.129339,-0.073239,-0.072457,-0.082936
beta_time,-5.45054,,-5.411681,,,,,,,


In [119]:
results_base=model_base.estimate()
results_model_nestedvehicle.likelihood_ratio_test(results_base, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=26.605028595633485, threshold=3.841458820694124)

In [242]:
V_pref=V_nested
nest_pref=nests_vehicle
model_name_pref='Model_nested_vehicle'
result_pref=results_model_nestedvehicle
estimation_pref=table_nestedvehicle

### Market share

In [120]:
LPMC.head()

,trip_id,household_id,person_n,trip_n,travel_mode,purpose,fueltype,faretype,bus_scale,survey_year,...,pt_interchanges,dur_driving,cost_transit,cost_driving_fuel,cost_driving_ccharge,driving_traffic_percent,log_time_walk,log_time_cycle,log_time_pt,log_time_drive
0,18,4,0,0,3,3,6,5,0.0,1,...,0,0.196389,0.0,0.53,0.0,0.035361,-0.271153,-1.462456,-0.681548,-1.627658
1,20,5,1,0,4,3,1,5,0.0,1,...,0,0.117222,0.0,0.41,0.0,0.097156,-0.513607,-1.696449,-0.812181,-2.143684
2,21,5,1,1,4,3,1,5,0.0,1,...,0,0.167778,0.0,0.46,0.0,0.243377,-0.505746,-1.586101,-0.974185,-1.785115
3,22,6,0,0,3,3,6,1,1.0,1,...,0,0.093889,1.5,0.23,0.0,0.204142,-1.072295,-2.061820,-1.416754,-2.365643
4,31,8,1,5,1,3,1,1,1.0,1,...,0,0.055000,1.5,0.14,0.0,0.075758,-1.749339,-2.905485,-1.972083,-2.900422


Individuals are uniquely defined by the pair (household_id, person_n) so we drop duplicates of this tupple to find the number of individuals in my samples

In [121]:
female_less_40=LPMC[(LPMC['female']==1) & (LPMC['age']<=40)]
temp=female_less_40[['household_id', 'person_n']].drop_duplicates()
N_S1=temp.shape[0]
N_S1

1302

In [122]:
female_more_40=LPMC[(LPMC['female']==1) & (LPMC['age']>40)]
temp=female_more_40[['household_id', 'person_n']].drop_duplicates()
N_S2=temp.shape[0]
N_S2

1112

In [123]:
male_less_40=LPMC[(LPMC['female']==0) & (LPMC['age']<=40)]
temp=male_less_40[['household_id', 'person_n']].drop_duplicates()
N_S3=temp.shape[0]
N_S3

1209

In [124]:
male_more_40=LPMC[(LPMC['female']==0) & (LPMC['age']>40)]
temp=male_more_40[['household_id', 'person_n']].drop_duplicates()
N_S4=temp.shape[0]
N_S4

1065

In [125]:
N=N_S1+N_S2+N_S3+N_S4

In [126]:
W_S1=N_S1/N
W_S2=N_S2/N
W_S3=N_S3/N
W_S4=N_S4/N

In [127]:
print(W_S1, W_S2, W_S3, W_S4)

0.27773037542662116 0.23720136518771331 0.2578924914675768 0.22717576791808874


In [128]:
print(np.round(W_S1,3), np.round(W_S2,3), np.round(W_S3,3), np.round(W_S4,3))

0.278 0.237 0.258 0.227


# Market Shares


We will there use the market_share function defined in lab session 03-indicators, but we adapted it to our models


Here we DO NOT simulate weights ? is it ok ?

## Market Shares with Modelpref

In [230]:
female=Variable('female')
age=Variable('age')
weights = database.DefineVariable('weights', W_S1*(age<=40)*(female==1) + W_S2*(age>40)*(female==1) +W_S3*(age<=40)*(female==0)+W_S3*(age>40)*(female==0))
weights=Variable('weights')

In [231]:
type(weights)
IndicatorTuple = namedtuple('IndicatorTuple', 'value lower upper') 

In [246]:
def market_share(utility):
    """Calculate the market shares of all alternatives, given the
    specification of the utility functions.

    :param utility: specification of the utility functions. It is a
        dict where the keys are the IDs of the alternatives, and the
        values are the expressions of the utility functions.
    :type utility: dict(int: biogeme.expressions.Expression)

    :return: a dictionary where each entry corresponds to an
        alternative, and associates its name with the InticatorTuple
        containing the value of the market share, and the lower and
        upper bounds of the 90% confidence interval.
    :rtype: dict(str: IndicatorTuple)

    """
    prob_WALK = models.lognested(V_pref, av, nests_vehicle, 1)
    prob_CYCLE = models.lognested(V_pref, av, nests_vehicle, 2)
    prob_PT = models.lognested(V_pref, av, nests_vehicle, 3)
    prob_DRIVE = models.lognested(V_pref, av, nests_vehicle, 4)
    """
    prob_WALK = models.loglogit(utility, None, 1)
    prob_CYCLE= models.loglogit(utility, None, 2)
    prob_PT = models.loglogit(utility, None, 3)
    prob_DRIVE = models.loglogit(utility, None, 4)
    """

    simulate = {
        'Prob. walk': prob_WALK,
        'Prob. cycle': prob_CYCLE,
        'Prob. PT': prob_PT,
        'Prob. drive': prob_DRIVE,
        'weights' : weights
        
    }
    biosim = bio.BIOGEME(database, simulate)
    simulated_values = biosim.simulate(results_model_nestedvehicle.getBetaValues())

    # We also calculate confidence intervals for the calculated quantities

    betas = biogeme_model_nestedvehicle.freeBetaNames()
    b = results_model_nestedvehicle.getBetasForSensitivityAnalysis(betas,useBootstrap=False)
    left, right = biosim.confidenceIntervals(b, 0.9)

    # Market shares are calculated using the weighted mean of the individual probabilities

    ## Alternative WALK

    simulated_values['Weighted prob. walk'] = simulated_values['weights'] * simulated_values['Prob. walk']
    left['Weighted prob. walk'] = left['weights'] * left['Prob. walk']
    right['Weighted prob. walk'] = right['weights'] * right['Prob. walk']

    marketShare_walk = simulated_values['Weighted prob. walk'].mean()
    marketShare_walk_left = left['Weighted prob. walk'].mean()
    marketShare_walk_right = right['Weighted prob. walk'].mean()
    
    
    ## Alternative cycle

    simulated_values['Weighted prob. cycle'] =simulated_values['weights'] * simulated_values['Prob. cycle']
    left['Weighted prob. cycle'] = left['weights'] * left['Prob. cycle']
    right['Weighted prob. cycle'] = right['weights'] * right['Prob. cycle']

    marketShare_cylce = simulated_values['Weighted prob. cycle'].mean()
    marketShare_cycle_left = left['Weighted prob. cycle'].mean()
    marketShare_cycle_right = right['Weighted prob. cycle'].mean()

    ## Alternative pt

    simulated_values['Weighted prob. PT'] = (
        simulated_values['weights'] * simulated_values['Prob. PT']
    )
    left['Weighted prob. PT'] = left['weights'] * left['Prob. PT']
    right['Weighted prob. PT'] = right['weights'] * right['Prob. PT']

    marketShare_PT = simulated_values['Weighted prob. PT'].mean()
    marketShare_PT_left = left['Weighted prob. PT'].mean()
    marketShare_PT_right = right['Weighted prob. PT'].mean()
    
                                                         
    ## Alternative drive

    simulated_values['Weighted prob. drive'] = (
        simulated_values['weights'] * simulated_values['Prob. drive']
    )
    left['Weighted prob. drive'] = left['weights'] * left['Prob. drive']
    right['Weighted prob. drive'] = right['weights'] * right['Prob. drive']

    marketShare_drive = simulated_values['Weighted prob. drive'].mean()
    marketShare_drive_left = left['Weighted prob. drive'].mean()
    marketShare_drive_right = right['Weighted prob. drive'].mean()

    return {
            'Walk': IndicatorTuple(
                value=marketShare_walk, 
                lower=marketShare_walk_left, 
                upper=marketShare_walk_right
            ),
            'Cycle': IndicatorTuple(
                value=marketShare_cylce, 
                lower=marketShare_cycle_left, 
                upper=marketShare_cycle_right
            ),
            'Public transportation': IndicatorTuple(
                value=marketShare_PT, 
                lower=marketShare_PT_left, 
                upper=marketShare_PT_right
            ),
            'Drive': IndicatorTuple(
                value=marketShare_drive, 
                lower=marketShare_drive_left, 
                upper=marketShare_drive_right
            )
    }

In [247]:
ms = market_share(V_nested)
for k, v in ms.items():
    print(
        f'Market share for {k}: {100*v.value:.1f}% '
        f'[{100*v.lower:.1f}%, '
        f'{100*v.upper:.1f}%]'
    )

100%|█████████████████████████████████████████| 100/100 [14:07<00:00,  8.47s/it]


Market share for Walk: -95.8% [-103.1%, -89.6%]
Market share for Cycle: -98.7% [-104.5%, -92.7%]
Market share for Public transportation: -36.5% [-38.8%, -34.2%]
Market share for Drive: -28.6% [-31.0%, -26.4%]


##  Market Shares with actual choice

In [254]:
count_walk=0
count_cycle=0
count_pt=0
count_drive=0
for c in LPMC['travel_mode']:
    if c==1:
        count_walk+=1
    if c==2:
        count_cycle+=1
    if c==3:
        count_pt+=1
    if c==4:
        count_drive+=1
print('Maket share for walk ' +  str(count_walk/5000))
print('Maket share for cycle ' +  str(count_cycle/5000))   
print('Maket share for pt ' +  str(count_pt/5000))   
print('Maket share for drive ' +  str(count_drive/5000))   

Maket share for walk 0.1714
Maket share for cycle 0.0294
Maket share for pt 0.3498
Maket share for drive 0.4494
